# Python Classes for Vector Calculus

This notebook overviews a Python library based on "Object structure of vector calculus", hereinafter *VC*. The central thrust of VC is that a mimetic approach to vector calculus, with computation imitating mathematics as closely as possible, is necessarily object-oriented. The essential objects are also structurally constrained.

The core mathematical types of vector calculus are normed vector spaces, vectors, linear operators or maps, and differentiable functions. My particular objective is mimetic expression of continuous optimization algorithms, which generally presume an inner product, so norms are assumed here to be inner product norms. This notebook reviews a Python realization of the structure explained in VC, with a few simple examples based on NumPy. 

## Spaces and Vectors

As explained in VC, (pre-Hilbert) vector spaces combine a set of data objects with two operations on them, linear combination and inner (dot) product. The set of data objects has infinite cardinality except in one obvious instance, so is not computationally realizable. However it is sufficient to be able to determine whether an object is a member of the set, and to obtain a new object in the set on request ("let $v \in V$"). These observations translate into four pseudo-code attributes of a Space object:
1. a boolean function that takes an object argument and returns True if the argument refers to a data object for this space;
2. a function with no arguments that returns a new data object;
3. a linear combination function that evaluates $y=ax + by$, with arguments $a$, $b$ (scalars), $x$, and $y$ (data objects). Error if either $x$ or $y$ is not a valid data object;
4. an inner product function that returns a scalar $\langle x, y \rangle$ for arguments $x$ and $y$. Error if either $x$ or $y$ is not a data object.

All Spaces have these attributes, so the collection of Spaces is naturally expressed as an abstract class. In Python,

In [ ]:
from abc import ABC, abstractmethod

class Space(ABC):
    @abstractmethod
    def isData(self,x):
        pass
    @abstractmethod
    def getData(self):
        pass
    @abstractmethod
    def linComb(self, a, x, y, b=1.0):
        pass
    @abstractmethod    
    def dot(self,x,y):
        pass
    ...

The *Space* class declaration in *vcl.py* includes several other convenient methods, including a self-description interface and a *cleanup* method to remove any part of a data object that Python garbage collection does not automatically remove.

The almost-universal mathematical vernacular calls the data objects of a vector space "vectors" As explained in VC, this is logically incorrect, but also misleading: data objects must be combined with the other attributes of their vector space to act functionally as vectors. So a vector is not a data object alone, but a composite of a data object *together with* a vector space with its linear combination and inner product functions, for which the data object belongs to its proper set of data objects.

While *vcl.Space* is an abstract type, asserting behaviour but not implementing it, every aspect of vector behaviour is determined by attributes of the corresponding space. So the Python realization *vcl.Vector* is a concrete, rather than abstract, class - its attributes are defined, not merely declared:


In [ ]:
class Vector:
    def __init__(self, sp):
        self.space = sp
        self.data = sp.getData()
    def __del__(self):
        self.space.cleanup(self.data)            
    def linComb(self,a,x,b=1.0):
        self.space.linComb(a,x.data,self.data,b)
    def dot(self,x):
        return self.space.dot(self.data,x.data)
    ...

The full definition in *vcl.py* includes a mechanism for assigning a *Vector* to an existing data object, rather than the new data object assigned to it on construction. This possibility is convenient in applications. Several other convenience attributes are also provided.

For a simple example, I will construct a vector space based on NumPy. This choice makes a point: NumPy is already a fine environment for matrix algebra. However it does not offer interfaces for functions on subsets of vector spaces, nor for expression of algorithms defined in terms of vector functions, such as Newton's method. NumPy's array manipulation capabilities support straightforward construction of a vector space type, in the form of a *Space* as defined (partly) above. The key choice is to us NumPy *ndarrays* as the data objects. Each space corresponds mathematically to ${\bf R}^n$, so is characterized by its dimension. So an object is a data object of an *npSpace* if it is an column *numpy.ndarray* of the right dimension. 

In [ ]:
import vcl
import numpy as np

# numpy vector space
class npSpace(vcl.Space):
    def __init__(self,n):
        self.dim = n
    def getData(self):
        return np.zeros(self.dim).reshape(self.dim,1)
    def isData(self,x):
        return (isinstance(x,np.ndarray) and x.shape() == (self.dim,1))
    def linComb(self,a,x,y,b=1.0):
        y = a*x + b*y
    def dot(self,x,y):
        return np.dot(x.T,y)[0][0]

*npSpace* is a sub- (or derived) class of *Space*, as indicated by the first line in the definition. So it can be used in any context that calls for a *Space*.

The full definition in *npvc.py* includes several other useful functions, that are (or could be) defined in terms of the core functions described above. The *lincomb* and *dot* functions (along with the others) are also written to provide error messages on failure.

Here is a very simple use of the *npSpace* and *Vector* classes:

In [ ]:
import numpy as np
import vcl
import npvc

dom = npvc.Space(2)
x=vcl.Vector(dom)
x.data[0]=1
x.data[1]=1
print('A vector in 2D NumPy-based Space:')
x.myNameIs()

This example, simple as it is, makes another important point. There are NumPy *Space*s (namely *npSpace*s), but there is no NumPy *Vector*: there are only *Vector*s in *npSpace*s. The vector concept does not need to be specialized: it is a "function" of the choice of space, and is otherwise completely defined.

## Linear Operators

The third fundamental concept of linear algebra, after vector space and vector, is that of linear map or operator. These are simply functions whose domains and ranges are vector spaces, and which satisfy the linearity condition. Since the vector spaces at issue are presumed to be inner product spaces, linear operators really come in adjoint pairs. A suitable Python abstract class defining the behaviour of linear operators is

In [ ]:
class LinearOperator:
    @abstractmethod
    def getDomain(self):
        pass
    @abstractmethod
    def getRange(self):
        pass
    @abstractmethod
    def applyFwd(self,x, y):
        pass
    @abstractmethod
    def applyAdj(self,x, y):
        pass        
    @abstractmethod
    def myNameIs(self):
        pass

Besides being a logical imperative, enabling the *LinearOperator* instance to return references to its domain and range is actually useful in the formulation of algorithms, as will be seen below.

The obvious linear operator class for *npSpace* performs matrix-vector multiplication:

In [ ]:
class MatrixOperator(vcl.LinearOperator):    
    def __init__(self,dom,rng,mat):
        self.dom = dom
        self.rng = rng
        self.mat = np.copy(mat)
        #....    
    def applyFwd(self,x, y):
        y.data = self.mat@x.data
    def applyAdj(self,x, y):
        y.data = self.mat.T@x.data

The elided code in the constructor checks that the *npSpaces* *dom* and *rng* have the number of columns of the *numpy.ndarray*, respectively its number of rows, as their dimensions. 

Note that the domain and range are passed as *Space* arguments, therefore stored as references to external *Space* objects that exist independently of the *MatrixOperator*, whereas the matrix argument is copied, internal to the object. This construction requires the error-checking just mentioned, but has several advantages. Obviously the domain and range spaces could also be copied, but that would be logically incorrect as well as inconvenient. With the *MatrixOperator* storing references to externally defined spaces, membership in those spaces can be verified by simple comparison: to know that a *vcl.Vector* x is a member of the domain, for example, simply check the value of *x.space==self.dom*. It is not enough to check that the input vector and the domain have the same dimension. For example, subclasses of *npvc.Space* could be equipped with units, whence only checking equality of dimensions could lead to dimensional errors. Insisting that the spaces involved should be *the same objects* avoids such egregious errors.

The matrix, on the other hand, is naturally internal data. This construction maintains the identity of the *MatrixOperator* even if the NumPy array passed to the constructor is subsequently changed.

Here is a simple example of matrix multiplication via the *npvc.MatrixOperator* class. The textual overhead from expressing matrix multiplication as the action of a linear operator (as compared to straight NumPy) is: 3 lines of code, out of 15.

In [ ]:
import numpy as np
import vcl
import npvc

# domain space and vector in it
dom = npvc.Space(2)
x=vcl.Vector(dom)
x.data[0]=1
x.data[1]=1

# range space and vector in it
rng = npvc.Space(3)
y=vcl.Vector(rng)

# 3 x 2 matrix - initialize as outer product
#mat=y.data@x.data.T
mat=np.zeros((3,2))
mat[0,0]=1
mat[0,1]=1
mat[1,1]=1

# matrix operator based on mat
matop=npvc.MatrixOperator(dom,rng,mat)

# matrix-vector product as matrix operator 
# application
matop.applyFwd(x,y)

print('Input vector x')
x.myNameIs()
print('Matrix Operator matop')
matop.myNameIs()
print('Output vector y = matop(x)')
y.myNameIs()

# Functions

Having constructed a class for linear operators (i.e. linear functions), it's obvious how to build a class for (possibly) non-linear functions. First, absent linearity there is no natural adjoint concept, so there is only a "forward" application function. Second, for differentiable functions a derivative (a linear operator) is another attribute. 

These considerations suggest a very simple abstract interface:

In [ ]:
class Function(ABC):
    @abstractmethod
    def getDomain(self):
        pass
    @abstractmethod
    def getRange(self):
        pass
    @abstractmethod
    def apply(self,x,y):
        pass
    # should return linear op
    @abstractmethod
    def deriv(self,x):
        pass
    @abstractmethod
    def myNameIs(self):
        pass

A simple NumPy-based example is provided as *npvc.OpExpl1*. It realizes the function $f: {\bf R}^2 \rightarrow {\bf R}^3$ given by 
$$
f((x_0,x_1)^T) = (x_0*x_1, -x_1+x_0^2, x_1^2)^T.
$$
Its code is written according the principles outlined above. For instance, the domain and range spaces are constructed externally to the function object, and passed to its constructor as arguments. Their dimensions are checked to be 2 and 3 respectively. The function object stores references to these externally defined spaces. The *apply* and *deriv* methods sanity-check their arguments. See the code for details.

In [ ]:
dom = npvc.Space(2)
rng = npvc.Space(3)
f = npvc.OpExpl1(dom,rng)
x = vcl.Vector(dom)
y = vcl.Vector(rng)
x.data[0]=1
x.data[1]=-2
print('input vector:')
x.myNameIs()
f.apply(x,y)
print('output of apply method:')
y.myNameIs()
dfx = f.deriv(x)
print('output of deriv method:')
dfx.myNameIs()

Of course linear functions (operators) are also functions, so really *LinearOperator* should subclass *Function*. The derivative is a linear operator, so the definition of *Function* might seem to depend of the definition of *LinearOperator* which in turn depends on *Function*. However it is possible to take advanage of Python's genericity to break this cyclic dependence: the return type of the *Function.deriv* method isn't specified, because return types are not specified in Python! So re-write the definition of *LinearOperator* as follows:

In [ ]:
class LinearOperator(Function):
    def apply(self,x, y):
        self.applyFwd(x,y)
    def deriv(self,x):
        return self
    @abstractmethod
    def applyFwd(self,x, y):
        pass
    @abstractmethod
    def applyAdj(self,x, y):
        pass 

That is, *LinearOperator* is *Function* with two abstract methods made concrete (defined), two abstract methods added, and the others inherited. Of course the *deriv* method returns *self*, since a linear operator is its own derivative.

## Jets

The *jet* concept solves an obvious consistency vs. efficiency problem involving the attributes of functions. Suppose that *f* is a *vcl,Function*, and *x* is a *vcl.Vector*. Record the function's value in the *vcl.Vector* *y*, and the derivative in the *vcl.LinearOperator* *df*. Some number of lines later in the program, access these same objects. Is there any guarantee that they are still related in the same way? In fact, there is not. If *x* is changed but the function is not re-evaluated, then the three objects have become inconsistent. The only way to make sure that this inconsistency does not occur appears to be re-computing the value and derivative each time they are accessed. That could amount to considerable wasted computational effort.

Jets, in the sense meant here, borrow a concept from differential geometry, and offer a way around this problem. There are several equivalent definitions, of which I cite the one most relevant to computation. The $k$-jet of a $C^{\infty}$ function $f$ at a point $x$ in its domain is the sequence of values $D^{\alpha}f(x)$ of $f$ and all of its derivatives of orders $|\alpha| \le k$. This definition suggests an obvious container class, implemented in $vcl.Jet$. For the time being, I have implemented only the 1-jet. The constructor arguments are the *vcl.Function* *f* and *vcl.Vector* *x*. The methods *getPoint*, *getValue*, and *getDeriv* return the computational analogues of $x$, $f(x)$, and $Df(x)$ respectively. These are stored as internal copies, so changing *x* after creation of *vcl.Jet(f,x)* does not change the return values of these methods. Thus an instance of *vcl.Jet* provides access to a consistent set of values.

An earlier implementation of this concept in the C++ library RVL used access control and the *const* keyword to prevent any violation of the jet's internal data, so really offered a guarantee that the return values of its methods are coherent. Such guarantees are impossible to provide in Python, which does not implement *const* and makes all class data public. So *vcl.Jet* really only provides guidance to help the programmer maintain the coherence of function values. Vector data is also private in RVL, and can only be altered through the action of a few specified function classes. This restriction makes a *versioning* system possible. The jet methods compare the version index of a vector with a recorded index to tell whether the vector had been altered, thus update values and derivatives automatically whenever necessary. So RVL implements the jet concept as *f(x) for variable x*. Such automation is impossible in a Python framework: the VCL user is responsible for updating *vcl.Jet* instances as needed. 

## Scalar Functions

Scalar-valued functions are central players in optimization, but the classes described so far do not encompass them. ${\bf R}$ can be identified with a 1-D vector space over the reals, but it is not *the same* as that vector space. This is even more so in the computational context: *float* objects are not interchangeable with 1-D *ndarray*s. In VCL, the latter are data of *Vector*s, not themselves *Vector*s. So there are a couple of conceptual layers between scalars and vectors, and scalar valued functions require a their own proper class.

The one feature of this class deserving of mention is the representation of the derivative by the gradient, its Riesz representer.

Because it occurs so often, I include a definition of the standard least-squares function
$$
J(x) = \|F(x)-y\|^2
$$ 
in which $F: X \rightarrow Y$ is a map between Hilbert spaces $X$ and $Y$, $y \in Y$, and $J: X \rightarrow {\bf R}$. 

(Of course, there's nothing "least" about it, but minimizing $J$ is the standard least-squares optimization problem, and the objective function is stuck with the same name, in the vernacular.)

In [ ]:
# function x -> 0.5*|f(x)-b|^2
class LeastSquares(vcl.ScalarFunction):
    def __init__(self,f,b):
        self.f = f
        self.b = b
    def raw_value(self,x):
        res=vcl.Vector(self.f.getRange())
        self.f.apply(f,res)
        res.linComb(-1,0,self.b)
        return 0.5*res.dot(res)
    def raw_gradient(self,x,g):
        res=vcl.Vector(self.f.getRange())
        self.f.apply(x,res)
        res.linComb(-1,0,self.b)
        df=self.f.deriv(x)
        df.applyAdj(res,g)

Here is an example using once again the function *npvc.OpExpl1*

In [ ]:
dom = npvc.Space(2)
rng = npvc.Space(3)
f = npvc.OpExpl1(dom,rng)
x = vcl.Vector(dom)
y = vcl.Vector(rng)
g = vcl.Vector(dom)
y.data[0]=3
y.data[1]=2
y.data[2]=-3
x.data[0]=1
x.data[1]=-2
J = vcl.LeastSquares(f,y)
J.myNameIs()
print('input vector x:')
x.myNameIs()
print('J.value(x)=' + str(J.value(x)))
J.gradient(x,g)
print('gradient at x = ')
g.myNameIs()

## Product Spaces and Partial Derivatives

Most scientific data lives in (Cartesian) product spaces, and they turn up in lots of other ways. The computational realization *ProductSpace* simply makes a list of *Space* objects behave as a *Space*. A *ProductSpace* data object is naturally a list of data objects, one for each *Space* factor. *ProductSpace* methods implement standard induced operations: linear combinations are lists of linear combinations, dot product is the sum of dot products, etc. The list of *Space* factors is available as the *spl* data member - which can be queried for length, individual factors, etc. since it's publicly accessible, like all class data in Python. 

In [ ]:
dom1=npvc.Space(1)
dom2=npvc.Space(1)
spacelist=[dom1,dom2]
pdom=vcl.ProductSpace(spacelist)
pdom.myNameIs()
x=vcl.Vector(pdom)
x.data[0][0]=1
x.data[1][0]=-2
x.myNameIs()

Comparison of this example with the preceding one reminds the reader that $R^2$ is not *the same* as $R^1 \oplus R^1$ - the two are isomorphic, but not identical. This mathematical truth is precisely reflected in the computational structure. Both examples construct a vector whose data array(s) has (have) two components with values -1 and 2, but in the first example it's an array reals of length 2, whereas in the second it's an array of length 2 of real arrays of length 1.

A further consequence of this reasoning: there is no such thing as a "product vector". Instead, there are vectors in product spaces. To allow access to items in the list of data objects of a vector in a product space as vectors in themselves, the *vcl.Vector* class provides a *component* method, implemented via the *link* method to wrap members of data list as *vcl.Vector*s. Altering a *component*'s data affects the parent *Vector* as you would expect: the *component* acts as a *view* into its parent *Vector*'s data

In [ ]:
x0=x.component(0)
x0.myNameIs()
x0.data[0]=2
x.myNameIs()

A function on a product space may (or may not) be provided with partial derivatives - that's one of those features that are implicit in the mathematical concept, but must be added explicitly to its computational homolog. If it is, the derivative is implemented as a *vcl.RowLinearOperator*, which provides a *vcl.LinearOperator* interface for a list of *vcl.LinearOperator*s. The individual operators making up a *RowLinearOperator* *A* are simply accessed by index from *A.oplist*.

I have modified *npvc.OpExpl1* to make the domain $R^1 \oplus R^1$ rather than $R^2$, and implemented the derivative as a *vcl.RowLinearOperator*. This change costs a couple of extra lines of code to build up the list of partial derivatives. Note that the output of the derivative, applied to a particular vector, is the same as the sum of the partial derivatives applied to its components, as it should be.

In [ ]:
f = npvc.OpExpl2(pdom,rng)
x.data[0][0]=1
x.data[1][0]=-2
print('input vector:')
x.myNameIs()
f.apply(x,y)
print('output of apply method:')
y.myNameIs()
dfx = f.deriv(x)
print('output of deriv method:')
dfx.myNameIs()
dx=vcl.Vector(pdom)
dx.data[0][0]=2
dx.data[1][0]=-3
dy=vcl.Vector(rng)
print('input to deriv.applyFwd')
dx.myNameIs()
dfx.applyFwd(dx,dy)
print('output of deriv.applyFwd')
dy.myNameIs()
dfx0 = f.partialDeriv(x,0)
cdx0 = dx.component(0)
dy0=vcl.Vector(rng)
dfx0.applyFwd(cdx0,dy0)
print('input of partial deriv 0')
cdx0.myNameIs()
print('output of partial deriv 0')
dy0.myNameIs()
dfx1 = f.partialDeriv(x,1)
cdx1 = dx.component(1)
dy1=vcl.Vector(rng)
dfx1.applyFwd(cdx1,dy1)
print('input of partial deriv 1')
cdx1.myNameIs()
print('output of partial deriv 1')
dy1.myNameIs()

## Algorithms

The main excuse for inventing this machinery is to make algorithms based on vector calculus easier to express. This section describes several examples.

### Conjugate Gradient Algorithm for Linear Least Squares

This algorithm is adapted from the Hestenes-Stiefel conjugate gradient algorithm for positive definite symmetric linear systems. It gives an approximate solution of the least-squares problem
$$
\mbox{Given }A\mbox{ and }b \in B, \,\min_{x \in X} \|Ax-b\|^2
$$ 
in which $X$ and $B$ are Hilbert spaces and $A: X \rightarrow B$ is a (bounded) linear operator. Assuming that $A$ is coercive (full column rank in the finite-dimensional case), $x$ is also the unique solution of the linear system (normal equation)
$$
A^TA x = A^Tb
$$
The conjugate gradient algorithm, as described in Golub and van Loan, sections 10.2 and 10.3; Nocedal and Wright, algorithm 5.2, can be applied directly to this system. The only difference between that algorithm and the one described here is the introduction of a vector variable ($q$ below) to avoid explicit construction of the normal operator $A^TA$. Hanke, section 2.3, describes essentially this algorithm.

As I have written it here, five auxiliary vectors are required: $r, p, s \in X$, and $e, q \in B$. At each iteration, $e=b-Ax$ is the residual vector, $r=A^T(b-Ax)$ the normal residual vector. The iteration terminates when the length of either of these two vectors falls below a factor $\epsilon, \rho \in (0,1)$ of its original length. 

The algorithm proceeds in two phases. In each of the following steps, the equality sign "$=$" represents assignment, that is, the right hand side is first evaluated, the overwritten on the left hand side.

Initialization:

1. $x = 0$
2. $e = b$
3. $r = A^Tb$
4. $p = r$
7. $\gamma_0 = \langle r, r \rangle_X$
8. $\gamma = \gamma_0$
9. $k=0$
    
Iteration: Repeat while $k<k_{\rm max}$, $\|e\|>\epsilon \|b\|$, and $\|r\|>\rho \|A^Tb\|$

1. $q = Ap$
2. $s = A^Tq$
3. $\alpha = \gamma / \langle q, q\rangle_B$
4. $x = x+\alpha p$
5. $e = e-\alpha q$
6. $r = r-\alpha s$
7. $\delta = \langle r, r \rangle_X$
8. $\beta = \delta / \gamma$
9. $p = r + \beta p$
10. $\gamma = \delta$
11. $k = k+1$

The translation into VCL code is straightforward. I pick out a few examples to illustrate how this goes, all from the iteration phase.

(step 1) $q = Ap$ becomes *A.applyFwd(p,q)*

(step 2) $s = A^Tq$ becomes *A.applyAdj(q,s)*

(step 3) $\alpha = \gamma / \langle q, q\rangle_B$ becomes *alpha = gamma/q.dot(q)*

(step 4) $x = x+\alpha p$ becomes *x.linComb(alpha,p)* (*linComb* is 
often called *axpy*, standing for "(y = ) a x plus y")

The complete algorithm (function *cg* in the module *vcalg*) includes several levels of screen output, from none to printing the norms of $e$ and $r$ at every iteration.

Properties of the algorithm are described in the cited references and many others. Key facts:

1. the residual (norm of $e$) decreases monotonically. 

2. the normal residual (norm of $r$) decreases eventually, but not monotonically (in general).

3. for a system of dimension $n$, in exact arithmetic, the algorithm terminates at a solution of the normal equations in $n$ or fewer iterations. In floating point arithmetic, the residual in the normal equations is generally on the order of the square root of macheps in $n$ iterations. For very large problems, useful convergence is governed by the the distribution of eigenvalues of $A^TA$, not by the dimension. 

To illustrate some of these features, I constructed a least squares problem using *MatrixOperator* with 4-dimensional domain and 6-dimensional range. I built noise free data and solved the corresponding least squares problem using *vcalg.cg*.

In [ ]:
import vcalg

# domain space and vector in it
dom = npvc.Space(4)
xstar=vcl.Vector(dom)
xstar.data[0]=1
xstar.data[1]=1
xstar.data[2]=1
xstar.data[3]=1

# range space and vector in it
rng = npvc.Space(6)
b=vcl.Vector(rng)

# 3 x 2 matrix - initialize as outer product
#mat=y.data@x.data.T
mat=np.zeros((6,4))
mat[0,0]=1
mat[1,1]=2
mat[2,2]=3
mat[3,3]=4

# matrix operator based on mat
matop=npvc.MatrixOperator(dom,rng,mat)

# initialize rhs
matop.applyFwd(xstar,b)

# solution, residual, normal residual vectors
x=vcl.Vector(dom)
e=vcl.Vector(rng)
r=vcl.Vector(dom)

# set cg parameters and run
kmax=20
eps=0.01
rho=0.01
vcalg.conjgrad(x=x, b=b, A=matop, kmax=kmax, eps=eps,\
               rho=rho, e=e, r=r, verbose=2)

# view result
print('\nsolution vector:')
x.myNameIs()

### Truncated Gauss-Newton Algorithm for Nonlinear Least Squares

Suppose that $F:U \rightarrow B$ is a (possibly nonlinear) function from an open subset $U$ of the Hilbert space $X$ to another Hilbert space $B$. "Nonlinear least squares" refers to the optimization problem,
$$
\mbox{Given }b \in B, \,\min_{x \in U} J(x),
$$ 
where $J(x)=0.5*\|F(x)-b\|^2$. Given a current estimate $x$ of the solution, Newton's method produces an update by using the gradient
$$
g(x) = DF(x)^T(F(x)-b)
$$
and Hessian
$$
H(x) = D(DF^T)(x)(F(x)-b) + DF(x)^TDF(x)
$$
by solving for the Newton step $s$: 
$$
x \leftarrow x+s; H(x)s = -g.
$$
The Gauss-Newton variant modifies the Hessian by dropping the first term. There are three reasons for doing this:

1. If the residual $F(x)-b$ is small at the solution (so that data has small noise), then this term should be small when $x$ is close to the minimizer;

2. Without that term, it is easy to see that $s$ is always a descent direction, assuming that $DF(x)$ has full column rank; and

3. Without that term, it is not necessary to compute the second derivative of $F$.

Also, the Gauss-Newton step is the solution of the linear least squares problem
$$
\min_s \|DF(x)s-g\|^2
$$
which suggests the possibility of computing $s$ via the Conjugate Gradient algorithm, which is particularly attractive for large-scale problems. Even better, it suggests a refinement for taking a partial step when far from the solution, where a full Newton step is not likely to be constructive, and furthermore reduces the total computational work. 

This refinement is due to Steihaug (see Nocedal and Wright, sections 4.1 and 6.4). Suppose that the quadratic model of $J$ based on the Gauss-Newton Hessian at $x$ is presumed to be sufficiently accurate in a ball $\{x+s:\|s\|\le \Delta\}$ that actual reduction in $J$ from taking the step is a significant fraction of the predicted reduction in $J$ based on the quadratic model. The step isn't allowed to exceed the "trust radius" $\Delta$.

The quadratic model around $x$ is
$$        
J(x+s)\approx J(x) + \langle s, g(x)\rangle  + 0.5\langle s, H(x)s\rangle
$$
where $H(x)=DF(x)^TDF(x)$. and the step $s$ solves $H(x)s = -g(x) = DF(x)^T(b-F(x))$ approximately. The predicted reduction is
$$
\mbox{predred} = J(x) - 0.5s^Tg(x)
$$
The actual reduction is
$$
\mbox{actred} = J(x) - J(x+s)
$$
Steihaug's algorithm uses the trust radius $\Delta$ as a maximum step length. The usual CG termination criterion (residual less than fraction of initial) is augmented by testing the step length: it is it greater than $\Delta$ then the iteration is stopped and the step is scaled back to have length $\Delta$. Otherwise the iteration terminates as usual. If this step $s$ of length at most $\Delta$, resulting from this modified CG, produces actual objective reduction (actred) that is too much less than predicted reduction (predred), then $\Delta$ is reduced and $s$ is re-computed. Otherwise, $x$ is updated to $x+s$. If the step is very successful, in that the actual reduction is close to the predicted reduction, than $\Delta$ is increased before then next update. 

As $x$ converges to a stationary point of $J$, eventually all steps are accepted. On the other hand, if the early iterates are far from a stationary point, $\Delta$ may be reduced so much that the CG stops at the first iteration: then $s$ is precisely the negative gradient, and a short enough step in that direction must produce actual reduction close to predicted reduction. Thus this "trust region" algorithm must converge to a stationary point from any initial guess, and ends with full Gauss-Newton steps.  

A single step of this algorithm involves an inner conjugate gradient iteration, and depends on a normal residual tolerance $0<\rho<1$, reduction ("Goldstein-Armijo") parameters $0<\gamma_{\rm red} < \gamma_{\rm inc} <1$, scaling parameters $\mu_{\rm red} < 1 < \mu_{\rm inc}$ satisfying $\mu_{\rm red}*\mu_{\rm inc}<1$, and gradient tolerance $0<\epsilon<1$. To update a current estimate $x$,

1. Apply the C-G algorithm to update $s$, starting at $s=0$. Stop when either (a) $\|H(x)s-g(x)\|<\rho\|g\|$, or (b) $\|s\| > \Delta$. In case (b), replace $s$ by $\Delta s /\|s\|$. Evaluate actred and predred as defined above.

2. if $\mbox{actred} < \gamma_{\rm red}*\mbox{predred}$, replace $\Delta$ by $\mu_{\rm red}*\Delta$ and repeat step 1.

3. Otherwise update $x$ (replace $x$ by $x+s$).

4. if $\mbox{actred} > \gamma_{\rm inc}*\mbox{predred}$, replace $\Delta$ by $\mu_{\rm inc}*\Delta$. 

The module *vcalg.py* contains a modified CG algorithm, and a truncated Gauss-Newton algorithm using it to implement the algorithm described here. Typical parameters for the various constants are $\rho=10^{-2}, \gamma_{\rm red}=0.1, \gamma_{\rm inc}=0.9, \mu_{\rm red}=0.5, \mu_{\rm inc}=1.8, \epsilon=10^{-2}$.

I apply this algorithm to the so-called Rosenbrock function, a moderately difficult nonlinear least-squares test problem (Nocedal and Wright, Excercise 9.1). The basic example is 2x2 and depends on a scale factor, usually set to 10. The function to be minimized is then 
$$
f(x_0,x_1) = 100(x_1-x_0^2)^2 + (1-x_0)^2
$$ 
which is equivalent to $J$ as defined above with
$$
F(x)=(10(x_1-x_0^2),-x_0)^T, \, b=(0,-1)
$$
I have doubled this function to make a 4x4 problem, with scale factors 10 and 2, that is,
$$
F(x)=(10(x_1-x_0^2),-x_0,2(x_3-x_2^2),-x_2)^T, \, b=(0,-1,0,-1)
$$
The minimum is at $(1,1,1,1)^T$, where $J=0$, so it is a global minimizer, also the unique stationary point of the Rosenbrock function.

This function is defined in *npvc.DoubleRosie*, as a function on the 4-dimensional *npvc.Space*. The *deriv* method returns a *npvc.MatrixOperator*, as in the other examples above.

I initialize $x$ at a point far from the global stationary point. The inner CG iteration is given enough iterations that it returns a very precise solution (in error by square root of macheps or less) of the Gauss-Newton equation $H(x)s + g(x)=0$ - if it is allowed to converge, instead of being stopped by the trust radius condition. I have suppressed the verbose output of the CG algorithm, to make the overall course of the GN iteration easier to see (cgverbose=0). For a large-scale problem, it will be important to monitor the behaviour of the CG iteration.

Note that the initial trust radius $\Delta=10$ is reduced four times before a successful step occurs, that is, $\mbox{actred} > \gamma_{\rm red}\mbox{predred}$.  Iteration 3 requires a further reduction of the trust radius, then the step is very successful ($\mbox{actred} > \gamma_{\rm inc}\mbox{predred}$, so the trust radius is increased. That turns out to be too long, so iteration 4 again decreases the trust radius, then takes another very successful step. This pattern repeats in Iterations 5 and 6. After iteration 6, all steps are successful.

The reader can change the verbosity level to see the actred/predred comparison that drives the modification of the trust radius (gnverbose=2), and display the details of the CG iterations (cgverbose=1 or 2) to see whether the usual convergence criterion or the trust radius condition is active. In fact only in the last few iterations does CG iteration run to completion: in previous G-N iterations, CG is truncated by the trust radius constraint.### Truncated Gauss-Newton Algorithm for Nonlinear Least Squares

In [ ]:
import vcl
import npvc
import vcalg

sp = npvc.Space(4)
x = vcl.Vector(sp)
b = vcl.Vector(sp)

x.data[0]=-1.2
x.data[1]=1.0
x.data[2]=-1.2
x.data[3]=1.0
b.data[0]=0.0
b.data[1]=-1.0
b.data[2]=0.0
b.data[3]=-1.0

F = npvc.DoubleRosie(sp)

vcalg.trgn(x, b, F, imax=40, eps=0.001, kmax=10, rho=1.e-6, \
           Delta=10.0, mured=0.5, muinc=1.8, \
           gammared=0.1, gammainc=0.95, \
           gnverbose=1, cgverbose=0)

print('\nsolution estimate:')
x.myNameIs()  

### Truncated Gauss-Newton Algorithm for Variable Projection Reduction of Separable Nonlinear Least Squares

Variable projection is a minimization algorithm for a scalar function of two (vector) variables $f(x_1,x_2)$, of class $C^2$. Suppose that for each $x_1$, $\tilde{x}_2(x_1)$ is a local minimizer of $x_2 \rightarrow f(x_1,x_2)$, and that $\tilde{x}_2$ is of class $C^1$. This is the case if, for example, the Hessian $D_2^2 f(x_1,\tilde{x}_2 (x_1))$ is positive definite symmetric. Define $\tilde{f}(x_1)=f(x_1,\tilde{x}_2 (x_1))$. It follows directly from the chain rule that if $x_1$ is a stationary point of $\tilde{f}$, then $(x_1,\tilde{x}_2(x_1))$ is a stationary point of $f$. Also, 
$$
\mbox{grad} \tilde{f}(x_1) = (\mbox{grad}_1 f)(x_1,x_2(x_1))
$$ 
(Golub and Pereyra, 1973).

If $f$ is the norm-squared of a vector-valued function, then so is $\tilde{f}$, and the Gauss-Newton algorithm is applicable.